In [1]:
import os
import torch
import torch.nn as nn
import numpy as np
from pandas import *
import torch.nn.functional as F
import dictionary_corpus
from torch.autograd import Variable
from model import RNNModel
from collections import defaultdict

In [2]:
torch.manual_seed(1111)
np.random.seed(1111)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
# LSTM model from (Kobzeva et al., 2022)
fn = "../data/lm/Norwegian/lstm-1.pt"
model_name = "LSTM-1"

In [4]:
model_ = None
with open(fn, "rb") as model_f:
    model_ = torch.load(fn, map_location=torch.device('cpu'))

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/serialization.py:868: SourceChangeWarning: source code of class 'model.RNNModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/serialization.py:868: SourceChangeWarning: source code of class 'torch.nn.modules.dropout.Dropout' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/serialization.py:868: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has ch

In [5]:
model = RNNModel("LSTM", 50001, 650, 650, 2, 0.2, False)
model.eval()
model.load_state_dict(model_.state_dict())

<All keys matched successfully>

In [6]:
# path to data files
data_path = "../data/lm/Norwegian"

In [7]:
dictionary = dictionary_corpus.Dictionary(data_path)

In [8]:
def check_vocab(word_list):
    """
    Check if elements from word_list are in the model's vocab
    """
    unknown = set()
    for w in word_list:
        try:
            idx = dictionary.word2idx[w]
        except KeyError:
            unknown.add(w)
    print(unknown)
    print(len(unknown), "word(s) is/are not in the model's vocabulary")

In [9]:
def sent_surprisal(prompt):
    """
    Assigns surprisal values to a sentence
    prompt: list with sentence tokens
    Returns a list with surprisal values for the sentence
    """
    # Sent has <eos> tokens at the start and end for which surprisal of 0 is assigned
    surprisal_arr = [0]  # surprisal for start <eos> token already added
    indices = [dictionary.word2idx[w] if w in dictionary.word2idx
               else dictionary.word2idx["<unk>"]
               for w in prompt]
    indices = torch.tensor(indices, dtype=torch.long)
    output, hidden = model(indices.view(-1, 1),  # Remember, (sequence_length, batch_size)
                           model.init_hidden(1))  # one input at a time, thus batch_size = 1
    for position, next_word in enumerate(prompt[1:-1]): # excluding actual surprisal for <eos>
        current_word_scores = output[position].view(-1)  # the output vector corresponding to the current word
        current_word_probs = F.log_softmax(current_word_scores, dim=0) # (log) softmax the score to get probabilities
        next_word_prob = current_word_probs[dictionary.word2idx[next_word]] # get the prob of the true next word
        surprisal = next_word_prob*(-1) 
        surprisal_arr.append(surprisal.item())
    surprisal_arr.append(0) # surprisal for end <eos> 
    return surprisal_arr

In [10]:
def get_surprisal_values(data):
    """
    Get surprisal values for a 'word' column in a df
    Returns a list with surprisal values for the whole df
    """
    surprisal_values = []
    # list with idx of rows that contain <eos> at the end of an item (ignores <eos> at the start)
    end_idx = data.loc[((data['word'] == '<eos>') & (data['region'] == 'end'))].index.to_list() 
    end_idx = [-1,*end_idx]  # inserting -1 as the start index to get the first sentence right
    for i in range(len(end_idx)-1):
        sent_range = range(end_idx[i]+1, end_idx[i+1]+1)
        sent_words = data.iloc[sent_range]['word'].to_list()
        surprisal_arr = sent_surprisal(sent_words)
        for s in surprisal_arr:
            surprisal_values.append(s)
    return surprisal_values

In [11]:
def filename_from_dataset(dataset):
    """
    Get output filename from input filename by adding "_result"
    """
    result_filename = '../data/results/lstm/' + os.path.basename(dataset)[:-4] + '_result.csv' 
    print(result_filename)
    return result_filename

In [12]:
def analyze_data(dataset, dependency):
    data = read_csv(dataset, index_col=0)
    words = data['word'].tolist()
    check_vocab(words)
    surprisal_values = get_surprisal_values(data)
    data["surprisal"] = surprisal_values
    data["dependency"] = dependency
    data["language"] = "Norwegian"
    result_fn = filename_from_dataset(dataset)
    data.to_csv(result_fn, encoding="utf-8-sig", index=False)

In [13]:
analyze_data('../data/test_sentences/eq_wh.csv', "Wh")
analyze_data('../data/test_sentences/whether_wh.csv', "Wh")
analyze_data('../data/test_sentences/subject_wh.csv', "Wh")
analyze_data('../data/test_sentences/unbound_wh.csv', "Wh")

set()
0 word(s) is/are not in the model's vocabulary
../data/results/lstm/eq_wh_result.csv
set()
0 word(s) is/are not in the model's vocabulary
../data/results/lstm/whether_wh_result.csv
set()
0 word(s) is/are not in the model's vocabulary
../data/results/lstm/subject_wh_result.csv
set()
0 word(s) is/are not in the model's vocabulary
../data/results/lstm/unbound_wh_result.csv


In [ ]:
analyze_data('../data/test_sentences/eq_rc.csv', "RC")
analyze_data('../data/test_sentences/whether_rc.csv', "RC")
analyze_data('../data/test_sentences/subject_rc.csv', "RC")
analyze_data('../data/test_sentences/unbound_rc.csv', "RC")

set()
0 word(s) is/are not in the model's vocabulary
